In [1]:
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
data = pd.read_csv("tmdb-movies.csv")

- Which genres are most popular from year to year? 
- What kinds of properties are associated with movies that have high revenues?
- Which director has the highest number of movies?
- What is the distribution of runtime?
- Which year has the highest number of movies?
- How many movies have a higher revenue_adj than budget_adj?
- Which movie has the longest runtime? the shortest?

In [3]:
data.columns

Index([u'id', u'imdb_id', u'popularity', u'budget', u'revenue',
       u'original_title', u'cast', u'homepage', u'director', u'tagline',
       u'keywords', u'overview', u'runtime', u'genres',
       u'production_companies', u'release_date', u'vote_count',
       u'vote_average', u'release_year', u'budget_adj', u'revenue_adj'],
      dtype='object')

In [18]:
reduced_columns = [
    u'popularity', 
    u'original_title', 
    u'director',
    u'runtime', 
    u'genres',
    u'release_date', 
    u'vote_count',
    u'vote_average', 
    u'release_year', 
    u'budget_adj', 
    u'revenue_adj',
]

data = data[reduced_columns]

In [20]:
data.head()

,popularity,original_title,director,runtime,genres,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,32.985763,Jurassic World,Colin Trevorrow,124,Action|Adventure|Science Fiction|Thriller,6/9/15,5562,6.5,2015,1.379999e+08,1.392446e+09
1,28.419936,Mad Max: Fury Road,George Miller,120,Action|Adventure|Science Fiction|Thriller,5/13/15,6185,7.1,2015,1.379999e+08,3.481613e+08
2,13.112507,Insurgent,Robert Schwentke,119,Adventure|Science Fiction|Thriller,3/18/15,2480,6.3,2015,1.012000e+08,2.716190e+08
3,11.173104,Star Wars: The Force Awakens,J.J. Abrams,136,Action|Adventure|Science Fiction|Fantasy,12/15/15,5292,7.5,2015,1.839999e+08,1.902723e+09
4,9.335014,Furious 7,James Wan,137,Action|Crime|Thriller,4/1/15,2947,7.3,2015,1.747999e+08,1.385749e+09


- parse genres: gather genres of all rows, split into a complete list of all genres, then apply one-hot encoding
- parse release_date into datetime

In [21]:
data.describe()

,popularity,runtime,vote_count,vote_average,release_year,budget_adj,revenue_adj
count,10866.000000,10866.000000,10866.000000,10866.000000,10866.000000,1.086600e+04,1.086600e+04
mean,0.646441,102.070863,217.389748,5.974922,2001.322658,1.755104e+07,5.136436e+07
std,1.000185,31.381405,575.619058,0.935142,12.812941,3.430616e+07,1.446325e+08
min,0.000065,0.000000,10.000000,1.500000,1960.000000,0.000000e+00,0.000000e+00
25%,0.207583,90.000000,17.000000,5.400000,1995.000000,0.000000e+00,0.000000e+00
50%,0.383856,99.000000,38.000000,6.000000,2006.000000,0.000000e+00,0.000000e+00
75%,0.713817,111.000000,145.750000,6.600000,2011.000000,2.085325e+07,3.369710e+07
max,32.985763,900.000000,9767.000000,9.200000,2015.000000,4.250000e+08,2.827124e+09


In [30]:
scott_movies = data[data["director"]=="Ridley Scott"]
#scott_movies

In [ ]:
set(data["release_year"]) # 1960-2015

In [50]:
from collections import Counter
genres = Counter()
genre_series = data["genres"].dropna()
for genre in genre_series:
    cur_genres = genre.split('|')
    for cur_genre in cur_genres:
        genres[cur_genre] += 1
genres.most_common()

[('Drama', 4761),
 ('Comedy', 3793),
 ('Thriller', 2908),
 ('Action', 2385),
 ('Romance', 1712),
 ('Horror', 1637),
 ('Adventure', 1471),
 ('Crime', 1355),
 ('Family', 1231),
 ('Science Fiction', 1230),
 ('Fantasy', 916),
 ('Mystery', 810),
 ('Animation', 699),
 ('Documentary', 520),
 ('Music', 408),
 ('History', 334),
 ('War', 270),
 ('Foreign', 188),
 ('TV Movie', 167),
 ('Western', 165)]